# Word2Vec vs GloVe

# * Goal of This File:

##### 1. Import Libraries, Helper Functions, and Constants ==> Data Sourcing and Munging ==> Utility Functions ==> Loading the Data ==> Merging all Individual Files

##### 2. Comparison ==> Class Weights ==> Word2Vec + GloVe

##### 3. RNN with Word2Vec 

##### 4. RNN with GloVe 

##### 5. Conclusion ==> Results ==> Tensorboard

## (1) Import Libraries, Helper Functions and Load Constants

In [1]:
!pip install pandas numpy nltk scikit-learn wordcloud\
seaborn gensim tensorflow imblearn xgboost matplotlib unrar pyunpack more-itertools patool keras-tqdm > /dev/null


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
VAST = True

if VAST:
    !sudo apt-get install unrar
    !sudo apt-get install rar

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unrar is already the newest version (1:5.6.6-2build1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
rar is already the newest version (2:5.5.0-1build1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [3]:
from nlp_ai_utils import *
from chunks_urls import CHUNKS_URLS
from updating_values import *
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

2023-05-10 12:10:21.891416: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
TF_ENABLE_ONEDNN_OPTS = 0
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
DATA_URLS = CHUNKS_URLS
GLOVE_URL = 'https://f005.backblazeb2.com/file/glove-embeddings-dims/glove.6B.100d.txt'
READY_DATA_URL = 'https://f005.backblazeb2.com/file/yelp-dataset-ready-for-models/ready_for_models.rar'
CLASS_WEIGHTS_URL = 'https://f005.backblazeb2.com/file/yelp-dataset-ready-for-models/class_weights.pickle'
UNIQUE_WORDS_URL = 'https://f005.backblazeb2.com/file/yelp-dataset-ready-for-models/unique_words.pickle'
LIMIT = DATA_LIMIT
TRAINED_MODELS_COUNT = TRAINED_MODELS
LARGER_DATASET_PATH = "../larger_dataset"
PREPROCESSED_CHUNKS_PATH = LARGER_DATASET_PATH + "/preprocessed_data_chunks/"
BASE_FILE_NAME = "chunk_"
FILE_FORMAT = ".csv"
DATA_IN_CHUNK = 116505
if LIMIT == 60:
    ACTUAL_DATA_SHAPE = DATA_IN_CHUNK * LIMIT + 20
else:
    ACTUAL_DATA_SHAPE = DATA_IN_CHUNK * LIMIT
RANDOM_STATE = CONST_RANDOM_STATE
READY_DATASET_PATH = "../larger_dataset/ready_for_models/"
USE_READY_DATA = False
USE_MODIFIED_LABELS = True

### 1.1 Data Sourcing and Munging

#### 1.1.1 Loading The Data

In [5]:
if not USE_READY_DATA:
    if not os.path.exists(LARGER_DATASET_PATH):
        os.mkdir(LARGER_DATASET_PATH)
    if not os.path.exists(PREPROCESSED_CHUNKS_PATH):
        os.mkdir(PREPROCESSED_CHUNKS_PATH)
    get_chunks(DATA_URLS,LIMIT,1,BASE_FILE_NAME,PREPROCESSED_CHUNKS_PATH,FILE_FORMAT)

chunk_1 already exists.
chunk_2 already exists.
chunk_3 already exists.
chunk_4 already exists.
chunk_5 already exists.
chunk_6 already exists.


#### 1.1.2 Merging all Individual Files

In [6]:
#get all names of downloaded files
if not USE_READY_DATA:
    all_file_names = get_all_file_names(BASE_FILE_NAME,LIMIT)

In [7]:
#read all chunks into a list
if not USE_READY_DATA:
    list_dfs = read_chunks(all_file_names,PREPROCESSED_CHUNKS_PATH,FILE_FORMAT)

In [8]:
#concatenate all chunks into a singular df
if not USE_READY_DATA:
    df = group_up_chunks(list_dfs)

In [9]:
#check how much of the data was actually downloaded
if not USE_READY_DATA:
    percent_loaded = check_no_missing_data(df.shape[0],ACTUAL_DATA_SHAPE)
    percent_loaded

In [10]:
if not USE_READY_DATA:
    review_data = df[['text', 'stars']].copy()

In [11]:
if not USE_READY_DATA:
    review_data.reset_index(inplace = True)
    review_data.drop(['index'],axis = 1,inplace = True)

In [12]:
if not USE_READY_DATA:
    review_data.rename(columns = {'text':'full_review_text','stars':'star_rating'}, inplace = True)

In [13]:
if not USE_READY_DATA:
    review_data.isnull().sum()

In [14]:
if not USE_READY_DATA:
    review_data.dropna(inplace = True)

In [15]:
if not USE_READY_DATA:
    review_data['full_review_text'].replace('', np.nan, inplace=True)
    review_data.dropna(inplace = True)

## (2) Comparison

In [16]:
if not USE_READY_DATA:
    X = review_data['full_review_text']
    y = review_data['star_rating']

In [17]:
if not USE_READY_DATA:
    X = pd.Series([str(text) for text in X])

In [18]:
if USE_MODIFIED_LABELS:
    translated_labels = translate_labels(y)
    y = pd.Series(encode_sent(translated_labels))

In [19]:
if not USE_READY_DATA:
    y = [label - 1 for label in y]

### 2.0 Classes' Shapes Explanation

#### Classes' shapes before and after (if USE_MODIFIED_LABELS = True):

BEFORE: 1 ==> Very Bad ==> AFTER: 2 ==> Negative Sentiment

BEFORE: 2 ==> Bad ==> AFTER: 2  ==> Negative Sentiment

BEFORE: 3 ==> Ok ==> AFTER: 3  ==> Neutral Sentiment

BEFORE: 4 ==> Good ==> AFTER: 1  ==> Positive Sentiment

BEFORE: 5 ==> Very Good ==> AFTER: 1  ==> Positive Sentiment

THEN: 1 is subtracted from each label. So, the Labels go FROM: 2,3,1 TO: 1,2,0

#### Classes' shapes before and after (if USE_MODIFIED_LABELS = False):

1 is subtracted from each label. So, the Labels go FROM: 1,2,3,4,5 TO: 0,1,2,3,4

Why is 1 subracted from each label? To bring the data into the preferred shape of the class weights in both sklearn and keras (starting label is 0, not 1).

### 2.1 Class Weights

In [20]:
if not os.path.exists("../pickle_files"):
    os.mkdir("../pickle_files")

In [21]:
if USE_READY_DATA:
    get_chunks([CLASS_WEIGHTS_URL],0,1,'class_weights','../pickle_files/','.pickle',False)

In [22]:
if USE_READY_DATA:
    class_weights = pickle.load(open("../pickle_files/class_weights.pickle", "rb"))

In [23]:
if not USE_READY_DATA:
    y = pd.Series(y)
    class_weights = compute_class_weight(class_weight = "balanced",classes = np.unique(y),y=y)
    class_weights = dict(zip(np.unique(y), class_weights))

### 2.2 Word2Vec

In [24]:
if not os.path.exists("../pickle_files/w2v_model.model"):
    print("Creating Embedding from scratch.")
    w2v_model = Word2Vec(sentences=[nltk.word_tokenize(text) for text in X], vector_size=100, window=5, min_count=1, workers=4)
    w2v_model.save("../pickle_files/w2v_model.model")
else:
    print("Embedding found.")
    w2v_model = Word2Vec.load("../pickle_files/w2v_model.model")

Embedding found.


In [25]:
if os.path.exists("../pickle_files/w2v_model.model"):
    w2v_model = Word2Vec.load("../pickle_files/w2v_model.model")

In [26]:
word_index_gensim = {}

for i, word in enumerate(w2v_model.wv.key_to_index):
    word_index_gensim[word] = i

In [27]:
vocab_size = len(word_index_gensim)
embedding_dim_gensim = w2v_model.vector_size

embedding_matrix_gensim = np.zeros((vocab_size, embedding_dim_gensim))

for word, i in word_index_gensim.items():
    if word in w2v_model.wv.key_to_index:
        embedding_matrix_gensim[i] = w2v_model.wv.get_vector(word)

EMBEDDING_LAYER_WORD2VEC = Embedding(vocab_size,
                            embedding_dim_gensim,
                            weights=[embedding_matrix_gensim],
                            trainable=False)

### 2.3 GloVe

In [28]:
if not os.path.exists('../glove_files'):
    os.mkdir('../glove_files')

In [29]:
if not USE_READY_DATA:
    get_chunks([GLOVE_URL],0,1,"glove.6B.100d",'../glove_files/','.txt',False)

glove.6B.100d already exists.


In [30]:
if not USE_READY_DATA:
    print('Getting Unique Words...')
    UNIQUE_WORDS = set(' '.join(X).split())

Getting Unique Words...


In [31]:
if USE_READY_DATA:
    get_chunks([UNIQUE_WORDS_URL],0,1,'unique_words',"../pickle_files/",'.pickle',False)

In [32]:
if USE_READY_DATA:
    UNIQUE_WORDS = pickle.load(open("../pickle_files/unique_words.pickle", "rb"))

### 2.4 Hyperparameters

In [33]:
VOCAB_SIZE = len(UNIQUE_WORDS)
RNN_UNITS = 64
CONV_FILTERS = 32
CONV_KERNEL_SIZE = 3
DROPOUT_PERCENT = 0.2
DENSE_UNITS = 512
LABELS_COUNT = len(y.unique())
EMBEDDING_DIM = 100
MAX_TEXT_LEN = 200
TRUNC_TYPE = 'post'
PADDING_TYPE = 'post'
OOV_TOKEN = "<OOV>"
BATCH_SIZE = 32
EPOCHS = 15

### 2.5 Data Split

##### ==> here is a quick explaination of how the dataset will be split using a smaller sample example.
##### ==> dataset => 100
##### ==> train_set => tr_s (example: 80)
##### ==> valid_set => vs (example: 10)
##### ==> test_set => te_s (example: 10)
##### ==> t = tr_s (80) + vs (10)
##### ==> train_set = x[:80]
##### ==> valid_set = x[80:t]
##### ==> test_set = x[t:] why t? because => vs = ts

##### use this guideline if you are confused about how the train-validation-test split was done. Also, this is a future guide for me as well in case I forget.

* train_set_size = 6,990,280 * 0.8 = 5,592,224
* valid_set_size = 6,990,280 * 0.1 = 699,028
* train_plus_valid = 5,592,224 + 699,028 = 6,291,252

==> To Confirm: test_size = 6,990,280 - 6,291,252 = 699,028

* train_set = [0:5,592,224]
* train_labels = [0:5,592,224]
* validation_set = [5,592,224:6,291,252] ==> 699,028
* validation_labels = [5,592,224:6,291,252] ==> 699,028
* test_set = [6,291,252,6,990,280] ==> 699,028
* test_labels = [6,291,252,6,990,280] ==> 699,028

In [34]:
if not USE_READY_DATA:
    TRAIN_PERCENT = 0.8
    VALID_TEST_PERCENT = 0.1
    TRAIN_SIZE = int(ACTUAL_DATA_SHAPE * TRAIN_PERCENT)
    VALID_TEST_SIZE = int(ACTUAL_DATA_SHAPE * VALID_TEST_PERCENT)
    TOTAL_TEST_SIZE = TRAIN_SIZE + VALID_TEST_SIZE
    train_set = X[:TRAIN_SIZE]
    train_labels = y[:TRAIN_SIZE]
    validation_set = X[TRAIN_SIZE:TOTAL_TEST_SIZE]
    validation_labels = y[TRAIN_SIZE:TOTAL_TEST_SIZE]
    test_set = X[TOTAL_TEST_SIZE:]
    test_labels = y[TOTAL_TEST_SIZE:]

In [35]:
if not os.path.exists("../larger_dataset"):
    os.mkdir("../larger_dataset")

In [36]:
if not os.path.exists(READY_DATASET_PATH):
    os.mkdir(READY_DATASET_PATH)

In [37]:
if USE_READY_DATA:
    get_chunks([READY_DATA_URL],0,1,'ready_for_models',READY_DATASET_PATH,'.rar',False)
    Archive(os.path.join(READY_DATASET_PATH,"ready_for_models.rar")).extractall(READY_DATASET_PATH)

In [38]:
if USE_READY_DATA:
    train_set_padded = pickle.load(open(READY_DATASET_PATH + "train_set_padded.pickle", "rb"))
    train_labels = pickle.load(open(READY_DATASET_PATH + "train_labels.pickle", "rb"))
    valid_set_padded = pickle.load(open(READY_DATASET_PATH + "valid_set_padded.pickle", "rb"))
    validation_labels = pickle.load(open(READY_DATASET_PATH + "validation_labels.pickle", "rb"))
    test_set_padded = pickle.load(open(READY_DATASET_PATH + "test_set_padded.pickle", "rb"))
    test_labels = pickle.load(open(READY_DATASET_PATH + "test_labels.pickle", "rb"))
    EMBEDDING_LAYER = pickle.load(open(READY_DATASET_PATH + "EMBEDDING_LAYER.pickle", "rb"))

In [39]:
# NOTE: THIS CELL TAKES A WHILE TO RUN.
if not os.path.exists(READY_DATASET_PATH + "train_set_padded.pickle"):
    print("Tokenizing the Dataset...")
    tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token=OOV_TOKEN)
    tokenizer.fit_on_texts(train_set)
    words_to_index = tokenizer.word_index

In [40]:
# NOTE: THIS CELL TAKES A WHILE TO RUN.
if not os.path.exists(READY_DATASET_PATH + "train_set_padded.pickle"):
    print("Padding the Training Set.")
    train_sequences = tokenizer.texts_to_sequences(train_set)
    train_set_padded = pad_sequences(train_sequences, maxlen=MAX_TEXT_LEN, padding=PADDING_TYPE, truncating=TRUNC_TYPE)
    
if not os.path.exists(READY_DATASET_PATH + "valid_set_padded.pickle"):
    print("Padding the Validation Set.")
    valid_sequences = tokenizer.texts_to_sequences(validation_set)
    valid_set_padded = pad_sequences(valid_sequences, maxlen=MAX_TEXT_LEN, padding=PADDING_TYPE, truncating=TRUNC_TYPE)
    
if not os.path.exists(READY_DATASET_PATH + "test_set_padded.pickle"):
    print("Padding the Testing Set.")
    test_sequences = tokenizer.texts_to_sequences(test_set)
    test_set_padded = pad_sequences(test_sequences, maxlen=MAX_TEXT_LEN, padding=PADDING_TYPE, truncating=TRUNC_TYPE)

In [41]:
if not os.path.exists(READY_DATASET_PATH + "train_set_padded.pickle"):
    print("Re-formatting Train Set Shape.")
    train_set_padded = np.array(train_set_padded)
    
if not os.path.exists(READY_DATASET_PATH + "train_labels.pickle"):
    print("Re-formatting Train Labels Shape.")
    train_labels = np.array(train_labels)
    
if not os.path.exists(READY_DATASET_PATH + "valid_set_padded.pickle"): 
    print("Re-formatting Validation Set Shape.")
    valid_set_padded = np.array(valid_set_padded)

if not os.path.exists(READY_DATASET_PATH + "validation_labels.pickle"):
    print("Re-formatting Validation Labels Shape.")
    validation_labels = np.array(validation_labels)

if not os.path.exists(READY_DATASET_PATH + "test_set_padded.pickle"):
    print("Re-formatting Testing Set Shape.")
    test_set_padded = np.array(test_set_padded)

if not os.path.exists(READY_DATASET_PATH + "test_labels.pickle"):
    print("Re-formatting Testing Labels Shape.")
    test_labels = np.array(test_labels)

In [42]:
if not USE_READY_DATA:
    word_to_vec_map = read_glove_vector('../glove_files/glove.6B.100d.txt')

In [43]:
if not os.path.exists(READY_DATASET_PATH + "EMBEDDING_LAYER_GLOVE.pickle"):
    print("Generating Embedding From Scratch.")
    vocab_mapping = len(words_to_index)
    embed_vector_len = word_to_vec_map['moon'].shape[0]

    emb_matrix = np.zeros((vocab_mapping, embed_vector_len))

    for word, index in words_to_index.items():
        embedding_vector = word_to_vec_map.get(word)
        if embedding_vector is not None:
            emb_matrix[index, :] = embedding_vector

    EMBEDDING_LAYER_GLOVE = Embedding(input_dim=vocab_mapping,\
                                output_dim=embed_vector_len, input_length=MAX_TEXT_LEN, weights = [emb_matrix], trainable=True)

In [44]:
if os.path.exists(READY_DATASET_PATH + "train_set_padded.pickle"):
    print("Using Pickle File!")
    train_set_padded = pickle.load(open(READY_DATASET_PATH + "train_set_padded.pickle", "rb"))
else:
    pickle_out = open(READY_DATASET_PATH + "train_set_padded.pickle",'wb')
    pickle.dump(train_set_padded,pickle_out)
    pickle_out.close()
    
if os.path.exists(READY_DATASET_PATH + "train_labels.pickle"):
    print("Using Pickle File!")
    train_labels = pickle.load(open(READY_DATASET_PATH + "train_labels.pickle", "rb"))
else:
    pickle_out = open(READY_DATASET_PATH + "train_labels.pickle",'wb')
    pickle.dump(train_labels,pickle_out)
    pickle_out.close()

Using Pickle File!
Using Pickle File!


In [45]:
if os.path.exists(READY_DATASET_PATH + "valid_set_padded.pickle"):
    print("Using Pickle File!")
    valid_set_padded = pickle.load(open(READY_DATASET_PATH + "valid_set_padded.pickle", "rb"))
else:
    pickle_out = open(READY_DATASET_PATH + "valid_set_padded.pickle",'wb')
    pickle.dump(valid_set_padded,pickle_out)
    pickle_out.close()
    
if os.path.exists(READY_DATASET_PATH + "validation_labels.pickle"):
    print("Using Pickle File!")
    validation_labels = pickle.load(open(READY_DATASET_PATH + "validation_labels.pickle", "rb"))
else:
    pickle_out = open(READY_DATASET_PATH + "validation_labels.pickle",'wb')
    pickle.dump(validation_labels,pickle_out)
    pickle_out.close()

Using Pickle File!
Using Pickle File!


In [46]:
if os.path.exists(READY_DATASET_PATH + "test_set_padded.pickle"):
    print("Using Pickle File!")
    test_set_padded = pickle.load(open(READY_DATASET_PATH + "test_set_padded.pickle", "rb"))
else:
    pickle_out = open(READY_DATASET_PATH + "test_set_padded.pickle",'wb')
    pickle.dump(test_set_padded,pickle_out)
    pickle_out.close()
    
if os.path.exists(READY_DATASET_PATH + "test_labels.pickle"):
    print("Using Pickle File!")
    test_labels = pickle.load(open(READY_DATASET_PATH + "test_labels.pickle", "rb"))
else:
    pickle_out = open(READY_DATASET_PATH + "test_labels.pickle",'wb')
    pickle.dump(test_labels,pickle_out)
    pickle_out.close()

Using Pickle File!
Using Pickle File!


In [47]:
if os.path.exists(READY_DATASET_PATH + "EMBEDDING_LAYER_GLOVE.pickle"):
    print("Using Pickle File!")
    EMBEDDING_LAYER_GLOVE = pickle.load(open(READY_DATASET_PATH + "EMBEDDING_LAYER_GLOVE.pickle", "rb"))
else:
    print("Pickling Embedding Layer!")
    pickle_out = open(READY_DATASET_PATH + "EMBEDDING_LAYER_GLOVE.pickle",'wb')
    pickle.dump(EMBEDDING_LAYER_GLOVE,pickle_out)
    pickle_out.close()

Using Pickle File!


## (3) RNN with Word2Vec

In [48]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

2023-05-10 12:10:40.990714: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-10 12:10:40.997967: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-10 12:10:40.998541: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [49]:
tf.keras.backend.clear_session()
tf.random.set_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
rnn_model_w2v = tf.keras.models.Sequential([
    EMBEDDING_LAYER_WORD2VEC,
    tf.keras.layers.Conv1D(filters=CONV_FILTERS, kernel_size=CONV_KERNEL_SIZE, padding='same', activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(keepdims = True),
    tf.keras.layers.SimpleRNN(RNN_UNITS),
    tf.keras.layers.Dense(DENSE_UNITS, activation = "relu"),
    tf.keras.layers.Dense(LABELS_COUNT, activation = "softmax")
])

2023-05-10 12:10:41.030704: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-10 12:10:41.031374: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-10 12:10:41.031727: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [50]:
rnn_model_w2v.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         25334700  
                                                                 
 conv1d (Conv1D)             (None, None, 32)          9632      
                                                                 
 global_max_pooling1d (Globa  (None, 1, 32)            0         
 lMaxPooling1D)                                                  
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                6208      
                                                                 
 dense (Dense)               (None, 512)               33280     
                                                                 
 dense_1 (Dense)             (None, 3)                 1539      
                                                        

In [51]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

In [52]:
metrics_callback = MetricsCallback(test_data = valid_set_padded, y_true = validation_labels)

In [53]:
rnn_model_w2v.compile(optimizer=Adam(), loss = SparseCategoricalCrossentropy(), metrics=['accuracy'])
rnn_model_w2v.fit(train_set_padded, train_labels,validation_data = (valid_set_padded,validation_labels),\
              batch_size=BATCH_SIZE, epochs=EPOCHS,class_weight=class_weights,\
              callbacks=[tensorboard_callback,metrics_callback,EarlyStopping(patience=4),ReduceLROnPlateau(factor=0.1, patience=2)])

Epoch 1/15


2023-05-10 12:10:43.478914: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-05-10 12:10:44.033625: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-10 12:10:44.035853: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x46ef83d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-10 12:10:44.035883: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-05-10 12:10:44.039363: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-10 12:10:44.140898: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the proce

2185/2185 [==============================] - 2s 1ms/step
              precision    recall  f1-score   support

           0       0.96      0.73      0.83     45732
           1       0.80      0.78      0.79     18313
           2       0.22      0.66      0.33      5858

    accuracy                           0.74     69903
   macro avg       0.66      0.72      0.65     69903
weighted avg       0.86      0.74      0.78     69903

Macro Weighted F1-Score: 0.7759539852696815
17476/17476 [==============================] - 82s 4ms/step - loss: 0.7279 - accuracy: 0.7090 - val_loss: 0.6173 - val_accuracy: 0.7352 - lr: 0.0010
Epoch 2/15
2185/2185 [==============================] - 2s 1ms/step
              precision    recall  f1-score   support

           0       0.96      0.74      0.84     45732
           1       0.83      0.76      0.79     18313
           2       0.23      0.70      0.34      5858

    accuracy                           0.74     69903
   macro avg       0.67      

In [54]:
if not os.path.exists("../saved_models"):
    os.mkdir("../saved_models")

In [56]:
TRAINED_MODELS_COUNT = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
rnn_model_w2v.save_weights("../saved_models/rnn_model_w2v_" + str(TRAINED_MODELS_COUNT) + ".h5")

In [58]:
pickle_out = open("../saved_models/rnn_model_w2v_params_" + str(TRAINED_MODELS_COUNT) + ".pickle",'wb')
pickle.dump({'EMBEDDING_DIM':EMBEDDING_DIM,'MAX_TEXT_LEN':MAX_TEXT_LEN,'BATCH_SIZE':BATCH_SIZE,'EPOCHS':EPOCHS,\
             'train_set_size':len(train_set),'RNN_UNITS':RNN_UNITS,\
             'CONV_FILTERS':CONV_FILTERS,'CONV_KERNEL_SIZE':CONV_KERNEL_SIZE,
             'chunks_used':(len(train_set) + len(validation_set) + len(test_set)) // DATA_IN_CHUNK,\
                'DENSE_UNITS':DENSE_UNITS,'LABELS_COUNT':LABELS_COUNT},pickle_out)
pickle_out.close()

## (4) RNN with GloVe

In [59]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [62]:
tf.keras.backend.clear_session()
tf.random.set_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
rnn_model_glove = tf.keras.models.Sequential([
    EMBEDDING_LAYER_GLOVE,
    tf.keras.layers.Conv1D(filters=CONV_FILTERS, kernel_size=CONV_KERNEL_SIZE, padding='same', activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(keepdims = True),
    tf.keras.layers.SimpleRNN(RNN_UNITS),
    tf.keras.layers.Dense(DENSE_UNITS, activation = "relu"),
    tf.keras.layers.Dense(LABELS_COUNT, activation = "softmax")
])

In [63]:
rnn_model_glove.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 100)          21927500  
                                                                 
 conv1d (Conv1D)             (None, 200, 32)           9632      
                                                                 
 global_max_pooling1d (Globa  (None, 1, 32)            0         
 lMaxPooling1D)                                                  
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                6208      
                                                                 
 dense (Dense)               (None, 512)               33280     
                                                                 
 dense_1 (Dense)             (None, 3)                 1539      
                                                        

In [64]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

In [65]:
metrics_callback = MetricsCallback(test_data = valid_set_padded, y_true = validation_labels)

In [66]:
rnn_model_glove.compile(optimizer=Adam(), loss = SparseCategoricalCrossentropy(), metrics=['accuracy'])
rnn_model_glove.fit(train_set_padded, train_labels,validation_data = (valid_set_padded,validation_labels),\
              batch_size=BATCH_SIZE, epochs=EPOCHS,class_weight=class_weights,\
              callbacks=[tensorboard_callback,metrics_callback,EarlyStopping(patience=4),ReduceLROnPlateau(factor=0.1, patience=2)])

Epoch 1/15
2185/2185 [==============================] - 3s 1ms/step
              precision    recall  f1-score   support

           0       0.98      0.77      0.87     45732
           1       0.88      0.85      0.86     18313
           2       0.26      0.72      0.38      5858

    accuracy                           0.79     69903
   macro avg       0.71      0.78      0.70     69903
weighted avg       0.89      0.79      0.82     69903

Macro Weighted F1-Score: 0.8245087694979837
17476/17476 [==============================] - 194s 11ms/step - loss: 0.5765 - accuracy: 0.7812 - val_loss: 0.5029 - val_accuracy: 0.7890 - lr: 0.0010
Epoch 2/15
2185/2185 [==============================] - 2s 1ms/step
              precision    recall  f1-score   support

           0       0.98      0.83      0.90     45732
           1       0.90      0.82      0.86     18313
           2       0.30      0.73      0.43      5858

    accuracy                           0.82     69903
   macro avg    

In [67]:
TRAINED_MODELS_COUNT = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
rnn_model_glove.save_weights("../saved_models/rnn_model_glove_" + str(TRAINED_MODELS_COUNT) + ".h5")

In [69]:
pickle_out = open("../saved_models/rnn_model_glove_params_" + str(TRAINED_MODELS_COUNT) + ".pickle",'wb')
pickle.dump({'EMBEDDING_DIM':EMBEDDING_DIM,'MAX_TEXT_LEN':MAX_TEXT_LEN,'BATCH_SIZE':BATCH_SIZE,'EPOCHS':EPOCHS,\
             'train_set_size':len(train_set),'RNN_UNITS':RNN_UNITS,\
             'CONV_FILTERS':CONV_FILTERS,'CONV_KERNEL_SIZE':CONV_KERNEL_SIZE,
             'chunks_used':(len(train_set) + len(validation_set) + len(test_set)) // DATA_IN_CHUNK,\
                'DENSE_UNITS':DENSE_UNITS,'LABELS_COUNT':LABELS_COUNT},pickle_out)
pickle_out.close()

## (5) Conclusion

In [70]:
if VAST:
    !tar -czf rnn_logs.tar.gz logs

### 5.1 Results

### 5.2 Tensorboard

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir logs/fit